In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import numpy as np
import math
import cv2
import requests

def plot_img(img):
    plt.imshow(img, cmap="gray")
    plt.xticks([])
    plt.yticks([])
    plt.show()
    
def svd(img, full_matrices=False):
    U, S, VT = np.linalg.svd(img, full_matrices=full_matrices)
    return (U, np.diag(S), VT)

def perc_storage(rank, n_rows, n_cols):
    original_space = n_rows*n_cols
    compressed_space = n_rows*rank + rank + n_cols*rank
    return compressed_space / original_space * 100


def perc_energy(S, r):
    return (np.trace(S[:r]) / np.trace(S)) * 100

def apply_svd_filter(img, k, filter_type):
    # Perform SVD on image matrix
    U, S, V = np.linalg.svd(img)
    
    # Apply Gram-Schmidt process to U
    Q = np.zeros_like(U)
    R = np.zeros((U.shape[1], U.shape[1]))
    for i in range(U.shape[1]):
        u = U[:, i]
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], U[:, i])
            u -= R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(u)
        Q[:, i] = u / R[i, i]
    
    # Create diagonal matrix from singular values
    S_k = np.diag(S[:k])
    
    # Reconstruct image using k singular values and compressed Q
    img_k = np.dot(Q[:,:k], np.dot(S_k, V[:k,:]))
    
    # Apply filter to reconstructed image
    if filter_type == "edge":
        img_k = np.abs(np.gradient(img_k)[0]) + np.abs(np.gradient(img_k)[1])
    elif filter_type == "denoise":
        img_k = np.round(img_k)
    
    return img_k.astype(np.uint8)



print('Select an Image:-')
print('1.Cat')
print('2.Panda')
print('3.Rabbit')
print('4.T-rex')
print('5.Parrot')


print()

a=int(input("Enter Your Choice:"))
print()

if a==1:
    image = imageio.imread("cat.jpg")
    print("Dimensions:",image.shape)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print("Real Image")
    
elif a==2:
    image = imageio.imread("panda.jpg")
    print("Dimensions:",image.shape)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print("Real Image")

elif a==3:
    image = imageio.imread("rabbit.jpg")
    print("Dimensions:",image.shape)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print("Real Image")
    
elif a==4:
    image = imageio.imread("Trex.jpg")
    print("Dimensions:",image.shape)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print("Real Image")
    
elif a==5:
    image = imageio.imread("parrot.jpg")
    print("Dimensions:",image.shape)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print("Real Image")
    
else:
    print('Enter a Valid Choice')


print()
print()
    

print("What Would You Like to do ?")
print("1.Resize and Grayscale Image")
print("2.Visulaise Compressed Images of Different Ranks")
print("3.Determine Space Required and Information Stored For a Particular Rank")
print("4.Denoising")
print("5.Edge Detection")
print("6.Contour")
print()

b=int(input("Enter Your Choice:"))
print()

if b==1:
    x=int(input("Enter X Co-ordinate:"))
    y=int(input("Enter Y Co-ordinate:"))
    print()
    
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (x,y))

    Rank=np.linalg.matrix_rank(IMG)
    U, S_mat, V_T = svd(IMG)
    n_rows, n_cols = IMG.shape

    print("Resizing and Grayscaling Image")
    print("Dimensions:",IMG.shape)
    print("Rank:",Rank)
    print('space required:',round(perc_storage(Rank, n_rows, n_cols), 2))
    print('information stored:',round(perc_energy(S_mat,Rank),2))
    plot_img(IMG)
    print("Resized and Grayscaled Image")
    
elif b==2:
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (300,400))
    U, S_mat, V_T = svd(IMG)
    print("Visualizing Compressed Images of Different ranks and Comparing them with Original Image.")
    print()

    n_rows, n_cols = IMG.shape
    RANKS = []
    for i in range(5):
        rank=int(input("Enter Rank:"))
        RANKS.append(rank)

    print()
    
    fig = plt.figure(0, (16, 10))

    for idx, r in enumerate(RANKS):
        X_r = U[:, :r] @ S_mat[:r, :r] @ V_T[:r, :]

        ax = plt.subplot(2,3, idx+1)
        ax.imshow(X_r, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])

        ax.set_title(f'''rank {r}\nspace required: {round(perc_storage(r, n_rows, n_cols), 2)}%
        information stored: {round(perc_energy(S_mat, r), 2)}''')

    ax = plt.subplot(2, 3, idx+2)
    ax.imshow(IMG, cmap='gray')
    ax.set_title('original image')
    ax.set_xticks([])
    ax.set_yticks([])

    plt.tight_layout()
    
elif b==3:
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (300,400))
    U, S_mat, V_T = svd(IMG)
    n_rows, n_cols = IMG.shape
    
    rank=int(input("Enter Rank:"))
    
    print('space required:',round(perc_storage(rank, n_rows, n_cols), 2))
    print('information stored:',round(perc_energy(S_mat,rank),2))
    
elif b==4:
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (300,400))
    img = np.asarray(IMG)
    denoising = apply_svd_filter(img, 50, "denoise")
    plt.imshow(denoising, cmap="gray")
    plt.title("Denoising")
    plt.axis('off')
    plt.show()
    
elif b==5:
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (300,400))
    img = np.asarray(IMG)
    edge_detection = apply_svd_filter(img, 10, "edge")
    plt.imshow(edge_detection, cmap="gray")
    plt.title("Edge Detection")
    plt.axis('off')
    plt.show()
    
elif b==6:
    IMG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    IMG = cv2.resize(IMG, (300,400))
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # Canny edges
    edged = cv2.Canny(gray, 30, 200)
    plt.title("Canny Edges")
    plt.imshow(edged)
    plt.axis('off')
    plt.show()
    
    # Finding Contours
    contour, hier = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print("Count of Contours  = " + str(len(contour)))
    
else:
    print("Enter a Valid Choice")
   
    
print()
print()
    

Select an Image:-
1.Cat
2.Panda
3.Rabbit
4.T-rex
5.Parrot



In [1]:
!pip install opencv-python


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2